<a href="https://colab.research.google.com/github/Jaeyoung-Choi/mnist_tutorial/blob/main/mnist_backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets as dsets
import torchvision.transforms as transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(42)
if device == 'cuda':
    torch.cuda.manual_seed_all(42)
    print(torch.cuda.current_device(), torch.cuda.device_count(), torch.cuda.get_device_name(0))

In [ ]:
def sigmoid(X):
    return 1 / (1 + torch.exp(-X))

def softmax(X):
    return torch.exp(X) / torch.sum(torch.exp(X), dim = 0)

In [ ]:
def dsigmoid(X):
    # return torch.exp(-X) / torch.pow(1 + torch.exp(-X), 2)
    return torch.mul(sigmoid(X), 1 - sigmoid(X))

def dsoftmax(X):
    #return torch.mul((torch.sum(torch.exp(X), dim = 1) - torch.exp(X)) / torch.pow(torch.sum(torch.exp(X), dim = 1), 2), torch.exp(X))
    return softmax(X) - torch.pow(softmax(X), 2)

In [ ]:
lr = 0.001

In [ ]:
class model:
    def __init__(self):
        self.W = [torch.normal(torch.zeros((784, 16)), torch.tensor([2])).to(device),
                  torch.normal(torch.zeros((16, 16)), torch.tensor([2])).to(device),
                  torch.normal(torch.zeros((16, 10)), torch.tensor([2])).to(device)]
        self.B = [torch.normal(torch.zeros((16)), torch.tensor(2)).to(device),
                  torch.normal(torch.zeros((16)), torch.tensor(2)).to(device),
                  torch.normal(torch.zeros((10)), torch.tensor(2)).to(device)]
    
    def forward(self, X):
        L1 = sigmoid(torch.matmul(X, self.W[0]) + self.B[0])
        L2 = sigmoid(torch.matmul(L1, self.W[1]) + self.B[1])
        Y = softmax(torch.matmul(L2, self.W[2]) + self.B[2])
        return Y

    def loss(self, X, Y):
        return torch.sum(torch.pow(self.forward(X) - Y, 2)) # / 원소 갯수(어케 구함?)
    
    def backward(self, X, Y):
        dW = [0, 0, 0]
        dB = [0, 0, 0]
        L1 = sigmoid(torch.matmul(X, self.W[0]) + self.B[0])
        L2 = sigmoid(torch.matmul(L1, self.W[1]) + self.B[1])
        FOR = softmax(torch.matmul(L2, self.W[2]) + self.B[2])

        dB[2] = torch.mul(dsoftmax(torch.matmul(L2, self.W[2]) + self.B[2]), 2 * (FOR - Y))
        # dW[2] = torch.mul(torch.reshape(L2, (16, 1)).expand(16, 10), dB[2].expand(16, 10)) #dB == dsoftmax
        dW[2] = torch.matmul(torch.transpose(torch.unsqueeze(L2, 0), 0, 1), torch.unsqueeze(dB[2], 0))

        # dB[1] = torch.mul(dsigmoid(torch.matmul(L1, self.W[1]) + self.B[1]), torch.sum(torch.transpose(dW[2], 0, 1), dim = 0))
        # dW[1] = torch.mul(torch.reshape(L1, (16, 1)).expand(16, 16), dB[1].expand(16, 16)) #dB == dsoftmax

        # dB[0] = torch.mul(dsigmoid(torch.matmul(X, self.W[0]) + self.B[0]), torch.sum(torch.transpose(dW[1], 0, 1), dim = 0))
        # dW[0] = torch.mul(torch.reshape(X, (784, 1)).expand(784, 16), dB[0].expand(784, 16)) #dB == dsoftmax

        # dB[1] = torch.mul(torch.reshape(torch.sum(torch.mul(self.W[2], dB[2].expand(16, 10)), dim = 1), (1, 16)), dsigmoid(torch.matmul(L1, self.W[1]) + self.B[1]))
        dB[1] = torch.mul(torch.matmul(dB[2], torch.transpose(self.W[2], 0, 1)), dsigmoid(torch.matmul(L1, self.W[1]) + self.B[1]))
        # dW[1] = torch.mul(torch.reshape(L1, (16, 1)).expand(16, 16), dB[1].expand(16, 16)) #dB == dsoftmax
        dW[1] = torch.matmul(torch.transpose(torch.unsqueeze(L1, 0), 0, 1), torch.unsqueeze(dB[1], 0))

        # dB[0] = torch.mul(torch.reshape(torch.sum(torch.mul(self.W[1], dB[1].expand(16, 16)), dim = 1), (1, 16)), dsigmoid(torch.matmul(X, self.W[0]) + self.B[0]))
        dB[0] = torch.mul(torch.matmul(dB[1], torch.transpose(self.W[1], 0, 1)), dsigmoid(torch.matmul(X, self.W[0]) + self.B[0]))
        # dW[0] = torch.mul(torch.reshape(X, (784, 1)).expand(784, 16), dB[0].expand(784, 16)) #dB == dsoftmax
        dW[0] = torch.matmul(torch.transpose(torch.unsqueeze(X, 0), 0, 1), torch.unsqueeze(dB[0], 0))


        self.W[2] = self.W[2] + torch.mul(dW[2], -1 * lr)
        self.B[2] = self.B[2] + torch.mul(dB[2], -1 * lr)

        self.W[1] = self.W[1] + torch.mul(dW[1], -1 * lr)
        self.B[1] = self.B[1] + torch.mul(dB[1], -1 * lr)

        self.W[0] = self.W[0] + torch.mul(dW[0], -1 * lr)
        self.B[0] = self.B[0] + torch.mul(dB[0], -1 * lr)

In [ ]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
def one_hot_encoding(x):
    arg = [0] * 10
    arg[x] = 1
    return arg

In [ ]:
mnist_test_X = []
for i in range(10000):
    mnist_test_X.append(mnist_test[i][0].reshape(1, 784))

mnist_test_X = torch.cat(mnist_test_X, dim = 0)
mnist_test_X = mnist_test_X.to(device)

In [ ]:
mnist_train_X = []
for i in range(60000):
    mnist_train_X.append(mnist_train[i][0].reshape(1, 784))

mnist_train_X = torch.cat(mnist_train_X, dim = 0)
mnist_train_X = mnist_train_X.to(device)

In [ ]:
mnist_test_Y = []
for i in range(10000):
    mnist_test_Y.append(one_hot_encoding(mnist_test[i][1]))
mnist_test_Y = torch.tensor(mnist_test_Y).to(device)

In [ ]:
mnist_train_Y = []
for i in range(60000):
    mnist_train_Y.append(one_hot_encoding(mnist_train[i][1]))
mnist_train_Y = torch.tensor(mnist_train_Y).to(device)

In [ ]:
M = model()

In [ ]:
M.W[0] = torch.load('/content/drive/MyDrive/mnist_train/88.3W0')
M.W[1] = torch.load('/content/drive/MyDrive/mnist_train/88.3W1')
M.W[2] = torch.load('/content/drive/MyDrive/mnist_train/88.3W2')

M.B[0] = torch.load('/content/drive/MyDrive/mnist_train/88.3B0')
M.B[1] = torch.load('/content/drive/MyDrive/mnist_train/88.3B1')
M.B[2] = torch.load('/content/drive/MyDrive/mnist_train/88.3B2')

In [ ]:
for tr in range(100):
    loss_sum = torch.tensor([0], dtype=torch.float).to(device)
    for i in range(60000):
        #print(tr, i, M.loss(mnist_train_X[i], mnist_train_Y[i]).to(torch.device('cpu')), end = '\t')
        M.backward(mnist_train_X[i], mnist_train_Y[i])
        loss_sum += M.loss(mnist_train_X[i], mnist_train_Y[i])
        #print(M.loss(mnist_train_X[i], mnist_train_Y[i]).to(torch.device('cpu')))
    print(tr, loss_sum)

0 tensor([21504.8633])
1 tensor([21348.1426])
2 tensor([21193.6914])
3 tensor([21041.8320])
4 tensor([20892.6211])
5 tensor([20745.3965])
6 tensor([20600.5430])
7 tensor([20457.7734])
8 tensor([20317.7539])
9 tensor([20180.2344])
10 tensor([20045.3184])
11 tensor([19913.1230])
12 tensor([19784.2539])
13 tensor([19658.9355])
14 tensor([19537.6914])
15 tensor([19420.6406])
16 tensor([19307.4355])
17 tensor([19197.3984])
18 tensor([19090.2695])
19 tensor([18985.6504])
20 tensor([18883.2793])
21 tensor([18783.6309])
22 tensor([18686.1777])
23 tensor([18591.2305])
24 tensor([18498.2969])
25 tensor([18407.4980])
26 tensor([18318.3398])
27 tensor([18230.5977])
28 tensor([18144.0469])
29 tensor([18058.4590])
30 tensor([17974.1777])
31 tensor([17891.0742])
32 tensor([17809.1172])
33 tensor([17728.5801])
34 tensor([17648.7148])
35 tensor([17570.2051])
36 tensor([17492.7207])
37 tensor([17416.2422])
38 tensor([17340.8594])
39 tensor([17266.7109])
40 tensor([17193.7246])
41 tensor([17121.9434])
42

In [ ]:
s_hit = 0
for i in range(10000):
    ans_y = torch.argmax(mnist_test_Y[i])
    exp_y = torch.argmax(M.forward(mnist_test_X[i]))
    print(ans_y, exp_y)
    if ans_y == exp_y:
        s_hit += 1

print(s_hit / 100)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
tensor(2) tensor(2)
tensor(8) tensor(8)
tensor(7) tensor(7)
tensor(4) tensor(4)
tensor(4) tensor(4)
tensor(0) tensor(0)
tensor(9) tensor(9)
tensor(3) tensor(3)
tensor(9) tensor(7)
tensor(5) tensor(5)
tensor(2) tensor(7)
tensor(6) tensor(6)
tensor(9) tensor(9)
tensor(1) tensor(1)
tensor(8) tensor(6)
tensor(6) tensor(6)
tensor(3) tensor(3)
tensor(6) tensor(6)
tensor(6) tensor(2)
tensor(8) tensor(8)
tensor(7) tensor(7)
tensor(8) tensor(8)
tensor(0) tensor(0)
tensor(4) tensor(4)
tensor(4) tensor(4)
tensor(8) tensor(8)
tensor(0) tensor(0)
tensor(3) tensor(5)
tensor(0) tensor(0)
tensor(0) tensor(0)
tensor(1) tensor(1)
tensor(2) tensor(2)
tensor(2) tensor(2)
tensor(7) tensor(9)
tensor(3) tensor(3)
tensor(1) tensor(1)
tensor(4) tensor(4)
tensor(3) tensor(3)
tensor(6) tensor(6)
tensor(2) tensor(2)
tensor(7) tensor(7)
tensor(5) tensor(2)
tensor(8) tensor(8)
tensor(2) tensor(3)
tensor(0) tensor(0)
tensor(1) tensor(1)
tensor(1) tensor(1)
tensor(0) tensor(0)
tens

In [ ]:
while True:
    for tr in range(100):
        for i in range(60000):
            #print(tr, i, M.loss(mnist_train_X[i], mnist_train_Y[i]).to(torch.device('cpu')), end = '\t')
            M.backward(mnist_train_X[i], mnist_train_Y[i])
            #print(M.loss(mnist_train_X[i], mnist_train_Y[i]).to(torch.device('cpu')))

    s_hit = 0
    for i in range(10000):
        ans_y = torch.argmax(mnist_test_Y[i])
        exp_y = torch.argmax(M.forward(mnist_test_X[i]))
        if ans_y == exp_y:
            s_hit += 1

    print(s_hit / 100)
    
    torch.save(M.W[0], '/content/drive/MyDrive/mnist_train/'+str(s_hit / 100)+'W0')
    torch.save(M.W[1], '/content/drive/MyDrive/mnist_train/'+str(s_hit / 100)+'W1')
    torch.save(M.W[2], '/content/drive/MyDrive/mnist_train/'+str(s_hit / 100)+'W2')

    torch.save(M.B[0], '/content/drive/MyDrive/mnist_train/'+str(s_hit / 100)+'B0')
    torch.save(M.B[1], '/content/drive/MyDrive/mnist_train/'+str(s_hit / 100)+'B1')
    torch.save(M.B[2], '/content/drive/MyDrive/mnist_train/'+str(s_hit / 100)+'B2')

88.83
88.99
89.21
89.3
89.24
89.28
89.28
89.16


KeyboardInterrupt: ignored

In [ ]:
torch.save(torch.tensor([0]), '/content/drive/MyDrive/mnist_train/'+'test.pt')